In [ ]:
# default_exp curator

# curator

> API details will follow

In [ ]:
# export
import pandas as pd
import numpy as np
from scipy.optimize import linprog

import logging
logger = logging.getLogger(__name__)

import unittest
tc = unittest.TestCase('__init__')


## Problem Definition
Our goal is to curate a subset from a general pool of samples, that will satisfy a list of conditions as close as possible.

The pool of samples is given in a dataframe, which we'll call *df_samples*, it has one row per sample, and the columns represent all sort of meta data and features of the samples.

Let's see an example:

In [ ]:
# Load dataframe from file.
df_samples = pd.read_csv('csvs/curation_pool.csv', 
                         converters={'age':int, 'birad':int})
df_samples = df_samples.set_index('study_id')
df_samples.head(10)

,exists,data_source,age,density,birad,lesion_type,largest_mass,is_pos
study_id,,,,,,,,
0,1,optimam,56,2,0,calcification,NaN,1
1,1,optimam,70,4,0,mass,16.87,1
2,1,optimam,70,2,0,mass,10.15,1
3,1,optimam,66,2,0,mass,10.71,1
4,1,imh,49,3,0,distortion,NaN,1
5,1,optimam,67,2,0,mass,9.24,1
6,1,optimam,47,4,0,mass,14.35,1
7,1,optimam,51,3,0,calcification,NaN,1
8,1,optimam,50,4,0,calcification,NaN,1


The conditions are given in a second dataframe, *df_cond_abs*. 
Each row of *df_cond_abs* is indexed by a *query* that can be applied to the df_samples (i.e. by using df_samples.query(query_string)). For each query the user specifies constraints supplied, regarding how many samples in the curated subset should satisfy the query. The constraints are given as a lower-bound and upper bound, as well as the penalty per violation (by default 1 if column not supplied). Ignore the *index_ref* column for now.

In [ ]:
# Get absolute numbers constraints 
df_cond_abs = pd.read_csv('csvs/curation_conditions_abs.csv').set_index('query')
df_cond_abs


,min,max,index_ref,penalty_per_violation
query,,,,
"is_pos == ""1""",400,400,-1,1
"is_pos == ""0""",400,400,-1,1
"data_source == ""optimam"" & is_pos == ""0""",160,240,-1,1
"data_source == ""imh"" & is_pos == ""0""",160,240,-1,1
"data_source == ""optimam"" & is_pos == ""1""",160,240,-1,1
"data_source == ""imh"" & is_pos == ""1""",160,240,-1,1
"lesion_type == ""mass"" & is_pos == ""1""",270,300,-1,1
"lesion_type == ""calcification"" & is_pos == ""1""",110,140,-1,1
"birad == ""1"" & is_pos == ""0""",300,320,-1,1


By applying the queries on the *df_samples* dataframe, we obtain df_bool, a boolean dataframe which has the samples as rows and the queries as columns. *df_bool* indicates which sample matches which query.

In [ ]:
# export
def get_query_features_df(df_samples, queries):
    """Apply queries on df_samples and return a boolean feature dataframe.
    
    If X = df_bool.values, then X[i, j] is True iff condition j is true for sample i
    """
    df_bool = df_samples[[]].copy()
    for query in queries:
        try:
            df_bool[query] = df_samples.eval(query)
        except Exception as e:
            print(query, e)
            raise        
    return df_bool


In [ ]:
df_bool = get_query_features_df(df_samples, df_cond_abs.index)
df_bool.head()


,"is_pos == ""1""","is_pos == ""0""","data_source == ""optimam"" & is_pos == ""0""","data_source == ""imh"" & is_pos == ""0""","data_source == ""optimam"" & is_pos == ""1""","data_source == ""imh"" & is_pos == ""1""","lesion_type == ""mass"" & is_pos == ""1""","lesion_type == ""calcification"" & is_pos == ""1""","birad == ""1"" & is_pos == ""0""","birad == ""2"" & is_pos == ""0""","lesion_type == ""mass"" & largest_mass<=10","lesion_type == ""mass"" & largest_mass>10 & largest_mass<=20","lesion_type == ""mass"" & largest_mass>20 & largest_mass<=50",age<50,age<60 & age>=50,age<70 & age>=60,age>=70
study_id,,,,,,,,,,,,,,,,,
0,True,False,False,False,True,False,False,True,False,False,False,False,False,False,True,False,False
1,True,False,False,False,True,False,True,False,False,False,False,True,False,False,False,False,True
2,True,False,False,False,True,False,True,False,False,False,False,True,False,False,False,False,True
3,True,False,False,False,True,False,True,False,False,False,False,True,False,False,False,True,False
4,True,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False


We can use this table to quickly see how many samples in our pool satisfy each query:

In [ ]:
df_bool.sum()

is_pos == "1"                                                 811
is_pos == "0"                                                 655
data_source == "optimam" & is_pos == "0"                      301
data_source == "imh" & is_pos == "0"                          354
data_source == "optimam" & is_pos == "1"                      653
data_source == "imh" & is_pos == "1"                          158
lesion_type == "mass" & is_pos == "1"                         556
lesion_type == "calcification" & is_pos == "1"                188
birad == "1" & is_pos == "0"                                  399
birad == "2" & is_pos == "0"                                  195
lesion_type == "mass" & largest_mass<=10                       58
lesion_type == "mass" & largest_mass>10 & largest_mass<=20    310
lesion_type == "mass" & largest_mass>20 & largest_mass<=50    178
age<50                                                        256
age<60 & age>=50                                              489
age<70 & a

## Basic definition, an Integer Program

Here we assume we have the boundaries given as absolute numbers
($a_i$=lower_bound, $b_i$=upper_bound per query $i$). 
This means we want to have at least $a_i$ samples and at most $b_i$ samples that will satisfy query number $i$.

Let $x_1, x_2, \ldots, x_M$ be our indications, which samples are included in the final set. Hence, $x_j = 1$ if sample $j$ is included in the final set, otherwise it is 0.

Let $A$ be a $MxN$ binary array such that $A_{ji} = 1$ if and only if query $i$ is true for sample $j$.
Therefore we have $N$ queries and $M$ samples.

The integer program representing this problem is:
$$
\begin{array}{lrcl}
\min &&& 1 \\
\text{s.t.}\hspace{0.5in}&&&&\\
&a_i &\leq& \left(\sum_{j: A_{ji}=1} x_j \right) \leq b_i  & \text{(for each query $i$)}\\
&&&x_j \in \{0, 1\}& \text{(for each sample $j$)}\\
\end{array}
$$

Notice:
1. This is an *integer* program, because the $x_j$'s are constrained to be integers.
2. There's no target function to optimize, we just need to find a feasible solution for the $x_j$'s.
3. We can actually rewrite the first constraint to be $ a \leq Ax \leq b$.

In [ ]:
df_cond = df_cond_abs.copy()
for q, row in df_cond.iterrows():
    row['max'] = 999
df_cond[3:4]['max']

query
data_source == "imh" & is_pos == "0"    999
Name: max, dtype: int64

In [ ]:
# export
class Curator(object):
    def __init__(self, df, df_cond, dedup=True, allow_violations=True):
        """Init a curator object.
        :param df: dataframe of samples (one row per samples).
        :param df_cond: dataframe of indexed by queries that can be applied to df, 
                        and the columns 'min', 'max', and 'index_ref' representing 
                        the required number of samples that should satisfy each query.
                        The column 'penalty_per_violation', if exist, indicates how much 
                        violation penalty would a single unit of violation in each query 
                        will cost (by default, 1)
        :param dedup: whether to combine rows that match the same set of queries.
                      If True (default), works faster, but slightly less accurate. 
        :param allow_violations: Allow infeasible solutions (should generally be True).
        """
                
        self.df_bool = get_query_features_df(df, df_cond.index)
        self.df_cond = df_cond.copy()
        self.dedup = dedup
        self.allow_violations = allow_violations

        if 'penalty_per_violation' not in self.df_cond:
            self.df_cond['penalty_per_violation'] = 1
        self.df_cond['index_ref'] = self.df_cond['index_ref'].astype('int')
        
        A = self.df_bool.values.astype('float').T # A.shape = [queries, samples]

        if dedup:
            A, self.ix, self.cnt = np.unique(A, return_inverse=True, return_counts=True, axis=1)
            # Multiply each (binary) column of A by the number of samples with those features.
            A = A * self.cnt
        else:
            self.cnt = 1

        self.n_constraints, self.n_samples = A.shape
        logger.info('#constraints=%d, #samples=%d' % A.shape)

        self.linprog_params = self.get_LP_params(A, self.df_cond)
        
        for key, val in self.linprog_params.items():
            logger.debug(key, np.shape(val))


    @staticmethod
    def get_abs_bounds(df_cond, cnt=None):
        """Convert bounds from relative fractions to absolute quantities.
           
           :param b: a matrix of shape (n_constraints, 3) where each row (l, u, j)
                     if j = -1:  the constraint is    "between l and u"
                     otherwise:  the constraint is    "between l*y_j and u*y_j"
                                 where y_j is the number of *included* samples that satisfy query j,
                                 given by the cnt parameter.
                                 
           :param cnt: how many *currently included* samples are satisfy each query.
                       if cnt is not None, y_j = cnt[j]. Otherwise, infer the bounds by looking
                       at the l and u values of b[j, :].
        """
        df_cond = df_cond.copy()
        cond_min = df_cond['min']
        cond_max = df_cond['max']
        for i, j in enumerate(df_cond['index_ref']):
            if j != -1:
                cond_min.iat[i] *= (cond_min.iat[j] if cnt is None else cnt[j])
                cond_max.iat[i] *= (cond_max.iat[j] if cnt is None else cnt[j])

        df_cond[['min', 'max']] = df_cond[['min', 'max']].round().astype('int')
        return df_cond

    def get_LP_params(self, A, df_cond):
        """Returns a dictionary with the arguments to scipy.optimize.linprog"""
        raise NotImplementedError

    def decode_solution(self, seed=None):
        """Returns a boolean vector of size n_samples, indicating chosen samples."""
        np.random.seed(seed or 0)
        x = self.solution.x[:self.n_samples] * self.cnt
        r = x % 1  # The remainder
        assert(np.abs(x.astype('int') + r - x).sum() < 1e-9, str(x[:5])+str(r[:5])+str(x[:5].astype('int')))
        if seed is None:
            r = r.round()
        else:
            r = np.random.rand(len(x)) <= r
 
        x = x.astype('int') + r.astype('int')

        if not self.dedup:
            # Original samples
            included = x            
        else:
            # x is number of samples to take from each group.            
            # Randomly choose from each group:
            included = np.zeros((len(self.ix),), dtype='bool')
            for g, cnt_g in enumerate(x):
                all_members = (self.ix == g).nonzero()[0]
                included_members = np.random.choice(all_members, cnt_g, replace=False)
                included[included_members] = True 

        self.included = included
        return included

    def get_summary(self, included):
        """Get summary of the queries, boundaries, and the violations."""
        
        cnt = self.df_bool[included.astype('bool')].sum()

        summary_df = self.get_abs_bounds(self.df_cond, cnt=cnt)
        summary_df['cnt'] = cnt
        summary_df['total'] = self.df_bool.sum()
        summary_df['violation'] = pd.DataFrame([summary_df['min'] - summary_df['cnt'],
                                                summary_df['cnt'] - summary_df['max']]).max().clip(0, None)
        print('Actual penalty:', summary_df['violation'].dot(summary_df['penalty_per_violation']), '  '
              'Total violations:', summary_df['violation'].sum())
        print('Included:', included.sum())

        return summary_df[['cnt', 'min', 'max', 'total', 'violation']]
        
    def run(self, method='revised simplex', seed=None):
        """Apply the LP. Use method='interior-point' for faster and less accurate solution."""
        
        included = summary_df = None
        self.solution = linprog(method=method, **self.linprog_params)
        logger.info(self.solution.message)
        
        if self.solution.success:
            included = self.decode_solution(seed=seed)
            print("Theoretical penalty:", self.solution.fun)
            summary_df = self.get_summary(included)
        else:
            logger.error("Could not find solution.")

        return included, summary_df

In [ ]:
# hide
from nbdev.showdoc import *
show_doc(Curator.__init__)
show_doc(Curator.get_abs_bounds)
show_doc(Curator.get_summary)
show_doc(Curator.run)
show_doc(Curator.decode_solution)
show_doc(Curator.get_LP_params)

<h4 id="Curator.__init__" class="doc_header"><code>Curator.__init__</code><a href="__main__.py#L3" class="source_link" style="float:right">[source]</a></h4>

> <code>Curator.__init__</code>(**`df`**, **`df_cond`**, **`dedup`**=*`True`*, **`allow_violations`**=*`True`*)

Init a curator object.
:param df: dataframe of samples (one row per samples).
:param df_cond: dataframe of indexed by queries that can be applied to df, 
                and the columns 'min', 'max', and 'index_ref' representing 
                the required number of samples that should satisfy each query.
                The column 'penalty_per_violation', if exist, indicates how much 
                violation penalty would a single unit of violation in each query 
                will cost (by default, 1)
:param dedup: whether to combine rows that match the same set of queries.
              If True (default), works faster, but slightly less accurate. 
:param allow_violations: Allow infeasible solutions (should generally be True).

<h4 id="Curator.get_abs_bounds" class="doc_header"><code>Curator.get_abs_bounds</code><a href="__main__.py#L44" class="source_link" style="float:right">[source]</a></h4>

> <code>Curator.get_abs_bounds</code>(**`df_cond`**, **`cnt`**=*`None`*)

Convert bounds from relative fractions to absolute quantities.

:param b: a matrix of shape (n_constraints, 3) where each row (l, u, j)
          if j = -1:  the constraint is    "between l and u"
          otherwise:  the constraint is    "between l*y_j and u*y_j"
                      where y_j is the number of *included* samples that satisfy query j,
                      given by the cnt parameter.
                      
:param cnt: how many *currently included* samples are satisfy each query.
            if cnt is not None, y_j = cnt[j]. Otherwise, infer the bounds by looking
            at the l and u values of b[j, :].

<h4 id="Curator.get_summary" class="doc_header"><code>Curator.get_summary</code><a href="__main__.py#L101" class="source_link" style="float:right">[source]</a></h4>

> <code>Curator.get_summary</code>(**`included`**)

Get summary of the queries, boundaries, and the violations.

<h4 id="Curator.run" class="doc_header"><code>Curator.run</code><a href="__main__.py#L117" class="source_link" style="float:right">[source]</a></h4>

> <code>Curator.run</code>(**`method`**=*`'revised simplex'`*, **`seed`**=*`None`*)

Apply the LP. Use method='interior-point' for faster and less accurate solution.

<h4 id="Curator.decode_solution" class="doc_header"><code>Curator.decode_solution</code><a href="__main__.py#L73" class="source_link" style="float:right">[source]</a></h4>

> <code>Curator.decode_solution</code>(**`seed`**=*`None`*)

Returns a boolean vector of size n_samples, indicating chosen samples.

<h4 id="Curator.get_LP_params" class="doc_header"><code>Curator.get_LP_params</code><a href="__main__.py#L69" class="source_link" style="float:right">[source]</a></h4>

> <code>Curator.get_LP_params</code>(**`A`**, **`df_cond`**)

Returns a dictionary with the arguments to scipy.optimize.linprog

### De-duplication

By default, we're also using de-duping: In many cases samples would have identical feature vectors. Hence in such cases we collapse every *group* $j$ of samples with identical feature rows to a single row, and we denote by $m_j$ the number of samples in the group. However, this has an effect on the linear progam, since now every $x_j$ indicates the fraction of samples from the *group* $j$ that is going to be included in the curated set.

Hence, We need to replace every occurence of $$\left(\sum_{j: A_{ji}=1} x_j \right)$$ with $$\left(\sum_{j: A_{ji}=1} m_j x_j \right)$$

Alternatively we can multiply each row $j$ of $A$ by $m_j$.

### Decoding the Solution

The solution lies in the values given to the $x_j$ variables by the linear solver. However, these values could be anything between 0 and 1. If we did not use de-duping, the simplest way to decide if a sample is included is to round each $x_j$ to the nearest integer, thus obtaining a boolean value (0 or 1).

If we did use de-duping, then each $x_j$ represent the *fraction* of samples from *group* $j$ that should be included. Hence we need to do two things in order to decide which individual samples to include:
1. Set $n_j = \mbox{round}(x_j \cdot m_j)$, the number of group members to include.
   For example, if $x_j=0.7$ and $m_j$=4, then $n_j=3$.
2. Randomly choose $n_j$ members from the total $m_j$ members of group $j$.

## AbsBoundariesCurator: Relaxing to linear progam

To turn this into a *linear* program, we need to perform a few relaxations.
First, we will let our $x_j$'s become real numbers between 0 and 1 (we'll round them up or down later).
Second, we introduce auxiliary variables, $c_i$'s, one per query, that will measure the *violation* of each query.
Now our goal is to minimize the total number of violations, $c_1 + c_2 + ... + c_N$.

Of course, not all violations are created equal, the user might want to enforce a stronger penalty for violating some conditions over others.  If the user provides the penalty for violating for each constraint, our target method becomes $p_1c_1 + p_2c_2 + ... + p_Nc_N$, where $p_i$ is the penalty-per-violation of query $i$.

Our *linear* progam now looks like this:
$$
\begin{array}{lrcl}
\min &&&p_1c_1 + p_2c_2 + ... + p_Nc_N \\
\text{s.t.}\hspace{0.5in}&&&&\\
&a_i - c_i &\leq& \left(\sum_{j: A_{ji}=1} x_j \right) \leq b_i + c_i & \text{(for each query $i$)}\\
&0 &\leq& x_j \leq 1& \text{(for each sample $j$)}\\
&0 &\leq& c_i & \text{(for each query $i$)}
\end{array}
$$

Notice that in the first constraint can be broken into two constraints:
$$
\begin{array}{rl}
a_i \leq &\left(\sum_{j: A_{ji}=1} x_j \right) + c_i&\\
         &\left(\sum_{j: A_{ji}=1} x_j \right) - c_i &\leq b_i
\end{array}
$$
one of them is always true, and as we increase $c_i$ from 0, at some point the other condition also becomes true.

We can also re-write these constraints as upper-bound constrains, and in vector form:
$$
\begin{array}{rll}
Ax - c &\leq& b\\
-Ax -c &\leq& -a
\end{array}
$$

This is the problem that the class *AbsBoundariesCurator* defines.

In [ ]:
# export
class AbsBoundariesCurator(Curator):
    def get_LP_params(self, A, df_cond):
        n_constraints, n_samples = A.shape
        df_cond = self.get_abs_bounds(df_cond) # In case the user supplied relative bounds

        bounds = [(0, 1)] * n_samples
        c = [0] * n_samples

        # Upper bound
        b_ub = df_cond['max'].values
        b_lb = df_cond['min'].values
        b_ub = np.hstack((b_ub, -b_lb))

        A_ub = np.vstack([A, # A * x <= ub
                         -A])# A * x >= lb ==> -A * x <= -lb

        if self.allow_violations: # Support non-feasible scenarios (pay penalty)
            # Add a new variable for every constraint, representing the violation.
            bounds += [(0, None)] * n_constraints
            c += df_cond['penalty_per_violation'].tolist()
            
            # Update the constraints to allow violations by c
            C = np.eye(n_constraints)
            A_ub = np.hstack((A_ub, np.vstack((-C, -C))))
        
        return dict(c=c, A_ub=A_ub, b_ub=b_ub, bounds=bounds)    
    

Let's use the *AbsBoundariesCurator* to find a curated set:

In [ ]:
cc = AbsBoundariesCurator(df_samples, df_cond_abs)

# Note, we are using here the interior-point solver which is
# faster but less accurate than the default simplex solver.
included, summary = cc.run(method='interior-point')

# Let's sanity-check that we obtained a reasonable solution.
tc.assertAlmostEqual(cc.solution.fun, 4.0, places=3)
tc.assertGreater(included.sum(), 780)
tc.assertLess(included.sum(), 820)

# The summary shows how many were included from every query,
# and the total number of violations.
summary

Theoretical penalty: 4.000000001349921
Actual penalty: 5   Total violations: 5
Included: 799


,cnt,min,max,total,violation
query,,,,,
"is_pos == ""1""",399,400,400,811,1
"is_pos == ""0""",400,400,400,655,0
"data_source == ""optimam"" & is_pos == ""0""",161,160,240,301,0
"data_source == ""imh"" & is_pos == ""0""",239,160,240,354,0
"data_source == ""optimam"" & is_pos == ""1""",241,160,240,653,1
"data_source == ""imh"" & is_pos == ""1""",158,160,240,158,2
"lesion_type == ""mass"" & is_pos == ""1""",269,270,300,556,1
"lesion_type == ""calcification"" & is_pos == ""1""",111,110,140,188,0
"birad == ""1"" & is_pos == ""0""",303,300,320,399,0


As you can see above, the linear solver had 4 violations, but after we decoded the solution (round the $x_j$ values and decide which samples to include), there were 5 violations in total. The optimal LP target value is always going to be a lower bound on the *integer* progam target.  

Our curated set has 799 members instead of 800, specifically one extra positive. Also, we have one extra positive study from optimam, and 2 too few studies from imh.  In addition, we are missing one positive study of type 'mass'.

Let's say the this latter constraint about positive masses is more important than others. We *really* want this constraint to be satisfied.  We can tweak the optimization by giving a larger penalty for each violation of this constraint.  Say 5 penalty points vs. only 1 penalty for the other conditions.

In [ ]:
df_cond_abs['penalty_per_violation'] = 1
df_cond_abs.loc['lesion_type == "mass" & is_pos == "1"', 'penalty_per_violation'] = 5

cc = AbsBoundariesCurator(df_samples, df_cond_abs)
included, summary = cc.run(method='interior-point')

summary

Theoretical penalty: 3.9999999974722984
Actual penalty: 5   Total violations: 5
Included: 801


,cnt,min,max,total,violation
query,,,,,
"is_pos == ""1""",400,400,400,811,0
"is_pos == ""0""",401,400,400,655,1
"data_source == ""optimam"" & is_pos == ""0""",163,160,240,301,0
"data_source == ""imh"" & is_pos == ""0""",238,160,240,354,0
"data_source == ""optimam"" & is_pos == ""1""",242,160,240,653,2
"data_source == ""imh"" & is_pos == ""1""",158,160,240,158,2
"lesion_type == ""mass"" & is_pos == ""1""",270,270,300,556,0
"lesion_type == ""calcification"" & is_pos == ""1""",111,110,140,188,0
"birad == ""1"" & is_pos == ""0""",304,300,320,399,0


Goodie!  We still have 5 violations but now that positive-mass constraint is satisfied! (and we curated 801 samples instead of 800).

Now we can go back to the original samples dataframe, and add a new column indicating which samples would participate in the final set:

In [ ]:
df_samples['included'] = included
df_samples.head()

,exists,data_source,age,density,birad,lesion_type,largest_mass,is_pos,included
study_id,,,,,,,,,
0,1,optimam,56,2,0,calcification,NaN,1,True
1,1,optimam,70,4,0,mass,16.87,1,False
2,1,optimam,70,2,0,mass,10.15,1,True
3,1,optimam,66,2,0,mass,10.71,1,False
4,1,imh,49,3,0,distortion,NaN,1,True


## Using Relative bounds for the constraints

The fact that the condition boundaties are given in absolute integer numbers is actually a limitation:
Say we are willing to have some flexibility with regard to the number of negatives we curate (i.e. anything in the range 350-450 is fine), but within the chosen set of negatives, we would like 25% to be with birad=2. Since we don't know how many negatives we'll turn up with, there is no way to put a tight bound (in absolute numbers) on the number of birad=2 samples.

What we want is to be able to bound a query relative to the (yet unknown) number of samples that satisfy a previous query.  So an alternative way to provide boundaries is in the form of a *fraction* relative to the resulting set satisfying a different query.


In [ ]:
# Get relative fraction constraints
df_cond_rel = pd.read_csv('csvs/curation_conditions_rel.csv').set_index('query')
df_cond_rel.reset_index()


,query,min,max,index_ref
0,"exists == ""1""",800.00,800.00,-1
1,"is_pos == ""1""",0.40,0.60,0
2,"is_pos == ""0""",0.40,0.60,0
3,"data_source == ""optimam"" & is_pos == ""0""",0.40,0.60,2
4,"data_source == ""imh"" & is_pos == ""0""",0.40,0.60,2
5,"data_source == ""optimam"" & is_pos == ""1""",0.40,0.60,1
6,"data_source == ""imh"" & is_pos == ""1""",0.40,0.60,1
7,"lesion_type == ""mass"" & is_pos == ""1""",0.65,0.70,1
8,"lesion_type == ""calcification"" & is_pos == ""1""",0.30,0.35,1
9,"birad == ""1"" & is_pos == ""0""",0.75,0.80,2


Here, in line 10, we ask that the number of samples satisfying the query [*birad == "2" & is_pos == "0"*] would be at least 20% and no more than 25% of the samples satisfying query 2 [*is_pos == "0"*], as indicated by the column *index_ref*. This is how we were able to define a condition relevant to the negative set without knowing how many negative we'll have at the end!

We still have to ground the solution in some absolute number of desired sample, so we used integer boundaries for the first query above, simply by setting *index_ref=-1* (otherwise the solution is not well defined and the LP solver might not converge).

Now how can we incorporate those constraints in a linear program?

### RelBoundariesCurator: 

So we need a new LP, and now we assume that each query $i$ has 3 values: $(a_i=\mbox{lower bound}, b_i=\mbox{upper bound}, k_i)$, where $k_i < i$.  BUT here the upper and lower bounds are *given as fractions* of the number of elements satisfying query $k_i$.

To simplify notations, we add additional auxiliary variables, $y_i$'s, that just count how many samples are included that satisfy each query. Hence we "constrain" $y_i$ to be $\left(\sum_{j: A_{ji}=1} x_j \right)$, or in other words, define $y = Ax$.

So, if $k_i = -1$, then use $a_i$ and $b_i$ as before (boundaries in absolute numbers).

If $k_i \geq  0$, then the boundaries in absolute numbers are $(a_i \cdot y_{k_i}, b_i \cdot y_{k_i})$, where 
$y_{k_i}$ is the (yet unknown) number of elements satisfying query $k_i$ in the solution.

This allows having constraints like "At least half of the positives should be under 50" and still allow flexibility in the number of positives. 

Out *linear* progam now looks like this (after conversion to upper-bound constraints):
$$
\begin{array}{lrcl}
\min &&&p_1c_1 + p_2c_2 + ... + p_Nc_N \\
\text{s.t.}\hspace{0.5in}&&&&\\
&y_i = \left(\sum_{j: A_{ji}=1} x_j \right) \\
&y_i - c_i - b_i \cdot y_{k_i} &\leq& 0 & \text{(for each query $i$)}\\
&-y_i - c_i + a_i \cdot y_{k_i} &\leq& 0 & \text{(for each query $i$)}\\
%&a_i \cdot y_{k_i} - c_i &\leq& y_i \leq b_i \cdot y_{k_i} + c_i & \text{(for each query $i$)}\\
&0 &\leq& x_j \leq 1& \text{(for each sample $j$)}\\
&0 &\leq& c_i & \text{(for each query $i$)}
\end{array}
$$


This is the problem that the class *RelBoundariesCurator* defines.

In [ ]:
# export
class RelBoundariesCurator(Curator):
    def get_LP_params(self, A, df_cond):
        n_constraints, n_samples = A.shape
        
        #             X's                          Y's
        bounds = [(0, 1)] * n_samples + [(0, None)] * n_constraints
        c = [0] * (n_samples          +    n_constraints)

        # Equalities
        Y = np.eye(n_constraints)
        A_eq = np.hstack((A, -Y))
        b_eq = np.zeros((n_constraints,))

        # Upper bounds
        Y_ub = Y.copy()
        Y_lb = Y.copy()

        b_ub = df_cond['max'].values.copy()
        b_lb = df_cond['min'].values.copy()
        for i, j in enumerate(df_cond['index_ref']):
            if j != -1:
                Y_ub[i, j] = -b_ub[i]
                Y_lb[i, j] = -b_lb[i]
                b_ub[i] = b_lb[i] = 0                
        b_ub = np.hstack((b_ub, -b_lb))

        A_ub = np.zeros((n_constraints*2, n_samples))
        A_ub = np.hstack((A_ub, np.vstack((Y_ub, -Y_lb))))
        

        if self.allow_violations: # Support non-feasible scenarios (pay penalty)
            # Add a new variable for every constraint, representing the violation.
            bounds += [(0, None)] * n_constraints
            c += df_cond['penalty_per_violation'].tolist()

            # Update the constraints to allow violations by c
            C = np.eye(n_constraints)
            A_ub = np.hstack((A_ub, np.vstack((-C, -C))))
            A_eq = np.hstack((A_eq, np.zeros((n_constraints, n_constraints))))

        return dict(c=c, A_eq=A_eq, b_eq=b_eq, A_ub=A_ub, b_ub=b_ub, bounds=bounds)


Let's run the *RelBoundariesCurator* to solve this (here with the simplex method):

In [ ]:
cc = RelBoundariesCurator(df_samples, df_cond_rel)
included, summary = cc.run()

# Let's sanity-check that we obtained a reasonable solution.
tc.assertAlmostEqual(cc.solution.fun, 0, places=3)
tc.assertGreater(included.sum(), 780)
tc.assertLess(included.sum(), 820)
tc.assertEqual(summary.violation.sum(), 0)

summary

Theoretical penalty: 1.7763568394002505e-15
Actual penalty: 0   Total violations: 0
Included: 800


,cnt,min,max,total,violation
query,,,,,
"exists == ""1""",800,800,800,1466,0
"is_pos == ""1""",395,320,480,811,0
"is_pos == ""0""",405,320,480,655,0
"data_source == ""optimam"" & is_pos == ""0""",184,162,243,301,0
"data_source == ""imh"" & is_pos == ""0""",221,162,243,354,0
"data_source == ""optimam"" & is_pos == ""1""",237,158,237,653,0
"data_source == ""imh"" & is_pos == ""1""",158,158,237,158,0
"lesion_type == ""mass"" & is_pos == ""1""",259,257,276,556,0
"lesion_type == ""calcification"" & is_pos == ""1""",119,118,138,188,0


And we reached an optimal solution! 